In [1]:
%matplotlib inline

Plot OpenFOAM data {#openfoam_example}
==================


In [15]:
from __future__ import annotations

import pyvista
from pyvista import examples
import os
import random
import dash
from dash import dcc, html
from dash.dependencies import Input, Output


This example uses data from a lid-driven cavity flow. It is recommended
to use `pyvista.POpenFOAMReader`{.interpreted-text role="class"} for
reading OpenFOAM files for more control over reading data.

This example will only run correctly in versions of vtk\>=9.1.0. The
names of the patch arrays and resulting keys in the read mesh will be
different in prior versions.


In [3]:
print(os.getcwd())
#filename = examples.download_cavity(load=False)

filename = "/home/cbyers/Projects/OpenFOAM/case_files/breeder_unit5/foam.foam"
reader = pyvista.OpenFOAMReader(filename)

/home/cbyers/Projects/Cranfield/AdaptiveVis


OpenFOAM datasets include multiple sub-datasets including the internal
mesh and patches, typically boundaries. This can be inspected before
reading the data.


In [4]:
print(f"All patch names: {reader.patch_array_names}")
print(f"All patch status: {reader.all_patch_arrays_status}")

object_methods = [method_name for method_name in dir(reader)
                  if callable(getattr(reader, method_name))]

print(f"All reader methods: {object_methods}")

All patch names: ['internalMesh', 'group/wall', 'patch/walls', 'patch/inlet', 'patch/outlet']
All patch status: {'internalMesh': True, 'group/wall': True, 'patch/walls': True, 'patch/inlet': True, 'patch/outlet': True}
All reader methods: ['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_set_defaults', '_set_defaults_post', '_set_directory', '_set_filename', '_update_information', 'cell_array_status', 'disable_all_cell_arrays', 'disable_all_patch_arrays', 'disable_all_point_arrays', 'disable_cell_array', 'disable_patch_array', 'disable_point_array', 'enable_all_cell_arrays', 'enable_all_patch_arrays', 'enable_all_point_arrays', 'enable_cell_array', 'enable_patch_array', 'enable_point_array', 'hide_progress', 'patch_array_status', 'point_array_st

This data is represented as a `pyvista.MultiBlock`{.interpreted-text
role="class"} object. The internal mesh will be located in the top-level
MultiBlock mesh.


In [5]:
mesh = reader.read()
print(reader.cell_array_status)
print(f"Mesh patches: {mesh.keys()}")
internal_mesh = mesh["internalMesh"]  # or internal_mesh = mesh[0]


<bound method PointCellDataSelection.cell_array_status of OpenFOAMReader('/home/cbyers/Projects/OpenFOAM/case_files/breeder_unit5/foam.foam')>
Mesh patches: ['internalMesh', 'boundary']


In [6]:
print(internal_mesh.cell_data.keys())

#struct_mesh = internalmesh.cast_to_explicit_structured_grid()

['U', 'k', 'nut', 'omega', 'p']


In this case the internal mesh is a
`pyvista.UnstructuredGrid`{.interpreted-text role="class"}.


Additional Patch meshes are nested inside another MultiBlock mesh. The
name of the sub-level MultiBlock mesh depends on the vtk version.


In [7]:
boundaries = mesh["boundary"]
print(boundaries)
print(f"Boundaries patches: {boundaries.keys()}")
print(boundaries["walls"])
print(boundaries["inlet"])
print(boundaries["outlet"])

MultiBlock (0x75d2191f6320)
  N Blocks    3
  X Bounds    -39.000, 39.000
  Y Bounds    -38.999, 39.000
  Z Bounds    -520.000, -1.000
Boundaries patches: ['walls', 'inlet', 'outlet']
PolyData (0x75d2191f62c0)
  N Cells:    466117
  N Points:   233923
  N Strips:   0
  X Bounds:   -3.900e+01, 3.900e+01
  Y Bounds:   -3.900e+01, 3.900e+01
  Z Bounds:   -5.200e+02, -1.000e+00
  N Arrays:   10
PolyData (0x75d2191f62c0)
  N Cells:    618
  N Points:   460
  N Strips:   0
  X Bounds:   -7.984e+00, 7.984e+00
  Y Bounds:   -8.000e+00, 8.000e+00
  Z Bounds:   -5.200e+02, -5.200e+02
  N Arrays:   10
PolyData (0x75d2191f62c0)
  N Cells:    4520
  N Points:   3616
  N Strips:   0
  X Bounds:   -3.900e+01, 3.900e+01
  Y Bounds:   -3.900e+01, 3.900e+01
  Z Bounds:   -4.000e+02, -4.000e+02
  N Arrays:   10


The default in OpenFOAMReader is to translate the existing cell data to
point data. Therefore, the cell data arrays are duplicated in point
data.


In [8]:
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")
print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (6677390, 3)         VECTORS
    k                       float32    (6677390,)
    nut                     float32    (6677390,)
    omega                   float32    (6677390,)
    p                       float32    (6677390,)           SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (1991088, 3)         VECTORS
    k                       float32    (1991088,)
    nut                     float32    (1991088,)
    omega                   float32    (1991088,)
    p                       float32    (1991088,)           SCALARS


This behavior can be turned off if only cell data is required.


In [9]:
reader.cell_to_point_creation = False
internal_mesh = reader.read()["internalMesh"]
print("Cell Data:")
print(internal_mesh.cell_data)
print("\nPoint Data:")

print(internal_mesh.point_data)

Cell Data:
pyvista DataSetAttributes
Association     : CELL
Active Scalars  : p
Active Vectors  : U
Active Texture  : None
Active Normals  : None
Contains arrays :
    U                       float32    (6677390, 3)         VECTORS
    k                       float32    (6677390,)
    nut                     float32    (6677390,)
    omega                   float32    (6677390,)
    p                       float32    (6677390,)           SCALARS

Point Data:
pyvista DataSetAttributes
Association     : POINT
Active Scalars  : None
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays : None


Now we will read in all the data at the last time point.


In [10]:
print(f"Available Time Values: {reader.time_values}")
reader.set_active_time_value(1000.0)
reader.cell_to_point_creation = True  # Need point data for streamlines
mesh = reader.read()
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

Available Time Values: [0.0, 100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0]


In [16]:
# Initialize the Dash app
app = dash.Dash(__name__)

In [19]:
# Create a function to generate a PyVista plot and save it as an HTML file
def create_pyvista_plot():
    plotter = pv.Plotter(notebook=True)
    plotter.add_mesh(mesh, scalars="p")
    plotter.view_isometric()
    plotter.show()
    plotter.export_html('plot.html')  # Export to an HTML file

In [20]:
app.layout = html.Div([
    html.H1("PyVista in Dash"),
    #dcc.Graph(id='3d-plot')
     html.Iframe(src='plot.html', style={'width': '100%', 'height': '600px'})  # Embed the HTML file
])

In [21]:
if __name__ == '__main__':
	#app.run_server(port=8050, mode='external')
    app.run_server(debug=True)

In [18]:
plotter = pyvista.Plotter()
plotter.add_mesh(mesh, scalars="p", cmap='viridis')
plotter.view_isometric()
plotter.show()

Widget(value='<iframe src="http://localhost:40247/index.html?ui=P_0x75d25c6613f0_0&reconnect=auto" class="pyvi…